Import the libraries needed

In [3]:
import pandas as pd
#from pandas.compat import StringIO
from pandas.tseries.offsets import DateOffset

from datetime import datetime
import psutil


import numpy as np
import os
import pyodbc
pyodbc.drivers()

import csv

from sklearn.preprocessing import StandardScaler
import seaborn as sns

pd.options.display.max_rows = 1000

# For the Yahoo Finance api
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override() 


### Database connections and SQL to retrieve the various data needed.

In [4]:
SQL_server = 'WINPRDAI1572\CLESQLDW'
Reporting_database = 'CLE_Reporting'

pd.set_option('max_row', None)
#pd.options.display.max_rows = 1000

# Set a value for the home folder.
home_folder = "."

# Set values for the various paths.
input_path = home_folder + "\data"

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

2024-07-19 15:50:14.728742
svmem(total=16756752384, available=5651202048, percent=66.3, used=11105550336, free=5651202048)


#  Add in the Postcode data.

In [62]:
df_Horizon_All_Features['Address_Postcode'] = (
    pd.to_numeric(df_Horizon_All_Features['Address_Postcode'],
                  errors='coerce')
      .fillna(0)
    )
df_Horizon_All_Features['Address_Postcode'] = df_Horizon_All_Features['Address_Postcode'].astype(int)
df_Horizon_All_Features.dtypes

ContractNumber                object
ProdCat                       object
ProdCode                      object
PolicyStatus                  object
Switch_Flag                    int64
PricingVariant               float64
Has_UWP                      float64
Lump_Sum_Taxation              int64
Tax_Notification_Type         object
Insurance_Tax_Type            object
Main_Tax_Country              object
Secondary_Tax_Country         object
Taxed_Retirement_Income      float64
Policy_Fee_Type_Desc          object
Future_Risk                   object
ServicingBroker              float64
InitialBroker                float64
BrokerCategory                object
InitialRate                  float64
ReneewalRate                 float64
VIP_Type_Id                  float64
Marital_Status_Id            float64
Nationality_Code              object
IsSmoker                     float64
Politically_Exposed_Id       float64
Birth_Place                   object
Address_Postcode               int32
S

In [63]:
# giving directory name
filename = input_path + '\Postcode_State_Metadata.xlsx'

#df_Postcode = pd.read_excel(filename,sheet_name='Tabelle1')
df_Postcode = pd.read_csv(filename)
df_Postcode['Postal_Code'] = df_Postcode['Postal_Code'].astype(int)

print(df_Postcode.dtypes)
df_Postcode.head(2)

Postal_Code          int32
Postal_Code_Name    object
District            object
State               object
dtype: object


,Postal_Code,Postal_Code_Name,District,State
0,64754,Eberbach,Landkreis Rhein-Neckar-Kreis,Baden-W�rttemberg
1,68159,Mannheim,Mannheim,Baden-W�rttemberg


In [65]:
# Left Join the list of campaign contract numbers to the main dataset.
df_Horizon_All_Features = pd.merge(df_Horizon_All_Features, df_Postcode, left_on = ['Address_Postcode'], right_on = ['Postal_Code'] , how = 'left')

del df_Postcode
df_Horizon_All_Features.head(5)

,ContractNumber,ProdCat,ProdCode,PolicyStatus,Switch_Flag,PricingVariant,Has_UWP,Lump_Sum_Taxation,Tax_Notification_Type,Insurance_Tax_Type,...,Term_Remaining,CustomerAge_Start_Horizon,Postal_Code_x,Postal_Code_Name_x,District_x,State_x,Postal_Code_y,Postal_Code_Name_y,District_y,State_y
0,6074160W,XRPP,XRPP_2005V1,I,0,NaN,NaN,0,None,None,...,9.0,57.0,52072.0,Aachen,St�dteregion Aachen,North Rhine-Westphalia,52072.0,Aachen,St�dteregion Aachen,North Rhine-Westphalia
1,6074937L,XRPP,XRPP_2005V1,I,0,NaN,NaN,0,None,None,...,14.0,52.0,10823.0,Berlin,Berlin,Berlin,10823.0,Berlin,Berlin,Berlin
2,6074962M,XRPP,XRPP_2005V1,I,0,NaN,NaN,0,None,None,...,23.0,42.0,61440.0,Oberursel (Taunus),Landkreis Hochtaunuskreis,Hesse,61440.0,Oberursel (Taunus),Landkreis Hochtaunuskreis,Hesse
3,6075850C,XRPP,XRPP_2005V1,F,0,NaN,NaN,0,None,None,...,15.0,46.0,42719.0,Solingen,Solingen,North Rhine-Westphalia,42719.0,Solingen,Solingen,North Rhine-Westphalia
4,6100638P,XRPP,XRPP_2005V1,I,0,NaN,NaN,0,None,None,...,26.0,40.0,40789.0,Monheim am Rhein,Landkreis Mettmann,North Rhine-Westphalia,40789.0,Monheim am Rhein,Landkreis Mettmann,North Rhine-Westphalia


# 13. Add in the funds data.

In [66]:
# READ IN directly from DW
#def Get_Fund_Values(server, database):
#    conn = pyodbc.connect(
#         'DRIVER={ODBC Driver 17 for SQL Server};'
#         'SERVER='+server+';'
#         'DATABASE='+database+';'
#         'Trusted_Connection=yes;')
#  
#    query = """SELECT p.policy_Number
#      ,(SELECT SUM(Investment_Ratio) FROM CLT_Policy_Fund_Values f0 WHERE f0.Policy_Number = p.Policy_Number and  f0.Valuation_Date = '2022-07-29' AND f0.Fund_Sub_Type = 'Unsmoothed' AND f0.Fund_Type_id = '1') AS InvRatioF_0
#      ,(SELECT SUM(Investment_Ratio) FROM CLT_Policy_Fund_Values f1 WHERE f1.Policy_Number = p.Policy_Number and  f1.Valuation_Date = '2022-07-29' AND f1.Fund_Sub_Type = 'Unsmoothed' AND f1.Fund_Type_id = '5') AS InvRatioU_0
#      ,(SELECT SUM(Investment_Ratio) FROM CLT_Policy_Fund_Values f2 WHERE f2.Policy_Number = p.Policy_Number and  f2.Valuation_Date = '2022-10-28' AND f2.Fund_Sub_Type = 'Unsmoothed' AND f2.Fund_Type_id = '1') AS InvRatioF_1
#      ,(SELECT SUM(Investment_Ratio) FROM CLT_Policy_Fund_Values f3 WHERE f3.Policy_Number = p.Policy_Number and  f3.Valuation_Date = '2022-10-28' AND f3.Fund_Sub_Type = 'Unsmoothed' AND f3.Fund_Type_id = '5') AS InvRatioU_1
#      ,(SELECT SUM(Investment_Ratio) FROM CLT_Policy_Fund_Values f4 WHERE f4.Policy_Number = p.Policy_Number and  f4.Valuation_Date = '2023-01-31' AND f4.Fund_Sub_Type = 'Unsmoothed' AND f4.Fund_Type_id = '1') AS InvRatioF_2
#      ,(SELECT SUM(Investment_Ratio) FROM CLT_Policy_Fund_Values f5 WHERE f5.Policy_Number = p.Policy_Number and  f5.Valuation_Date = '2023-01-31' AND f5.Fund_Sub_Type = 'Unsmoothed' AND f5.Fund_Type_id = '5') AS InvRatioU_2
#      ,(SELECT SUM(Investment_Ratio) FROM CLT_Policy_Fund_Values f6 WHERE f6.Policy_Number = p.Policy_Number and  f6.Valuation_Date = '2023-04-28' AND f6.Fund_Sub_Type = 'Unsmoothed' AND f6.Fund_Type_id = '1') AS InvRatioF_3
#      ,(SELECT SUM(Investment_Ratio) FROM CLT_Policy_Fund_Values f7 WHERE f7.Policy_Number = p.Policy_Number and  f7.Valuation_Date = '2023-04-28' AND f7.Fund_Sub_Type = 'Unsmoothed' AND f7.Fund_Type_id = '5') AS InvRatioU_3
#      ,(SELECT SUM(Investment_Ratio) FROM CLT_Policy_Fund_Values f8 WHERE f8.Policy_Number = p.Policy_Number and  f8.Valuation_Date = '2023-07-31' AND f8.Fund_Sub_Type = 'Unsmoothed' AND f8.Fund_Type_id = '1') AS InvRatioF_4
#      ,(SELECT SUM(Investment_Ratio) FROM CLT_Policy_Fund_Values f9 WHERE f9.Policy_Number = p.Policy_Number and  f9.Valuation_Date = '2023-07-31' AND f9.Fund_Sub_Type = 'Unsmoothed' AND f9.Fund_Type_id = '5') AS InvRatioU_4
#FROM CLT_Policy p
#INNER JOIN CLT_Product prd ON prd.Product_Id = p.Product_Id AND prd.Product_Category_Code IN ('GNP','GNPP');"""    
#    
#    
#    return pd.read_sql(query, conn)

## Call the SQL proc to get all  policies.
#df_Fund_Values = Get_Fund_Values(SQL_server, Reporting_database)
#df_Fund_Values.head(5)


# 14. Add in Sinus Milieus data

In [71]:
# giving directory name
filename = input_path + '\sinus_milieu.csv'

df_Sinus_Milieus = pd.read_csv(filename)

print(df_Sinus_Milieus.dtypes)
df_Sinus_Milieus.head(2)

CONTRACT_NUMBER                 object
DATA_VERSION                   float64
POLICY_NUMBER                   object
VN_MILIEU_CODE                  object
VP_MILIEU_CODE                  object
VN_TARGET_GROUP_CODE            object
VP_TARGET_GROUP_CODE            object
VN_MILIEU_DESCRIPTION           object
VN_TARGET_GROUP_DESCRIPTION     object
VP_MILIEU_DESCRIPTION           object
VP_TARGET_GROUP_DESCRIPTION     object
dtype: object


,CONTRACT_NUMBER,DATA_VERSION,POLICY_NUMBER,VN_MILIEU_CODE,VP_MILIEU_CODE,VN_TARGET_GROUP_CODE,VP_TARGET_GROUP_CODE,VN_MILIEU_DESCRIPTION,VN_TARGET_GROUP_DESCRIPTION,VP_MILIEU_DESCRIPTION,VP_TARGET_GROUP_DESCRIPTION
0,6097825M,2023.0,6097825M,04_EPE,Expeditive,Selbstoptimierer,Selbstoptimierer,04_EPE,Expeditive,Selbstoptimierer,Selbstoptimierer
1,6098944M,2023.0,6098944M,10_NOK,Neo-Ökologische,Selbstoptimierer,Selbstoptimierer,10_NOK,Neo-Ökologische,Selbstoptimierer,Selbstoptimierer


## Drop obsolette or repeated features.

In [74]:
df_Sinus_Milieus = df_Sinus_Milieus.drop(['DATA_VERSION','POLICY_NUMBER','VP_MILIEU_CODE','VP_TARGET_GROUP_CODE','VN_TARGET_GROUP_DESCRIPTION','VP_TARGET_GROUP_DESCRIPTION'],axis=1)


KeyError: "['DATA_VERSION' 'POLICY_NUMBER' 'VP_MILIEU_CODE' 'VP_TARGET_GROUP_CODE'\n 'VN_TARGET_GROUP_DESCRIPTION' 'VP_TARGET_GROUP_DESCRIPTION'] not found in axis"

In [75]:
df_Horizon_All_Features = pd.merge(df_Horizon_All_Features, df_Sinus_Milieus, left_on = ['ContractNumber'], right_on = ['CONTRACT_NUMBER'] , how = 'left') 
df_Horizon_All_Features.head(10)

MemoryError: Unable to allocate 47.2 MiB for an array with shape (6186041,) and data type int64

In [77]:
%who_ls DataFrame 

['df_Horizon_All_Features', 'df_Sinus_Milieus']

In [76]:
print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')


2024-07-18 19:53:23.304180
svmem(total=16756752384, available=1416294400, percent=91.5, used=15340457984, free=1416294400)


In [32]:

del df_Sinus_Milieus

In [33]:
print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')


2024-07-18 19:34:15.945066
svmem(total=16756752384, available=7508250624, percent=55.2, used=9248501760, free=7508250624)


## Drop PII columns, unused columns and obfuscate PII where needed

ContractNumber                  object
ProdCat                         object
ProdCode                        object
PolicyStatus                    object
Switch_Flag                      int64
PricingVariant                 float64
Has_UWP                          int64
Tax_Notification_Type           object
Insurance_Tax_Type              object
Main_Tax_Country                object
Secondary_Tax_Country           object
Taxed_Retirement_Income        float64
Policy_Fee_Type_Desc            object
Future_Risk                     object
ServicingBroker                float64
InitialBroker                  float64
BrokerCategory                  object
InitialRate                    float64
ReneewalRate                   float64
Policy_Number                   object
VIP_Type_Id                    float64
Marital_Status_Id              float64
Nationality_Code                object
IsSmoker                       float64
Politically_Exposed_Id         float64
Birth_Place              

In [27]:
print(df_Horizon_All_Features['PricingVariant'].value_counts())

6.0       4377627
4.0         79978
5.0         50395
4015.0      40284
4001.0      32887
4004.0      26253
4017.0      23652
4002.0       8066
4005.0       7350
4003.0       4347
4016.0        324
4045.0         11
4036.0         11
4044.0          7
Name: PricingVariant, dtype: int64


In [36]:
df_Horizon_All_Features['Has_UWP'] = df_Horizon_All_Features['Has_UWP'].fillna(0).apply(np.int64)
print(df_Horizon_All_Features['Has_UWP'].value_counts())



1    4651192
0    1534849
Name: Has_UWP, dtype: int64


In [34]:
df_Horizon_All_Features['Lump_Sum_Taxation'] = df_Horizon_All_Features['Lump_Sum_Taxation'].fillna(0).apply(np.int64)
print(df_Horizon_All_Features['Lump_Sum_Taxation'].value_counts())
df_Horizon_All_Features = df_Horizon_All_Features.drop(['Lump_Sum_Taxation'],axis=1)

KeyError: "['Lump_Sum_Taxation'] not found in axis"

In [37]:
df_Horizon_All_Features['Tax_Notification_Type'] = df_Horizon_All_Features['Tax_Notification_Type'].astype(str)
print(df_Horizon_All_Features['Tax_Notification_Type'].value_counts())

df_Horizon_All_Features = df_Horizon_All_Features.drop(['Tax_Notification_Type'],axis=1)
print(df_Horizon_All_Features.dtypes)
df_Horizon_All_Features.head(20)

None    6186041
Name: Tax_Notification_Type, dtype: int64
ContractNumber                  object
ProdCat                         object
ProdCode                        object
PolicyStatus                    object
Switch_Flag                      int64
PricingVariant                 float64
Has_UWP                          int64
Insurance_Tax_Type              object
Main_Tax_Country                object
Secondary_Tax_Country           object
Taxed_Retirement_Income        float64
Policy_Fee_Type_Desc            object
Future_Risk                     object
ServicingBroker                float64
InitialBroker                  float64
BrokerCategory                  object
InitialRate                    float64
ReneewalRate                   float64
Policy_Number                   object
VIP_Type_Id                    float64
Marital_Status_Id              float64
Nationality_Code                object
IsSmoker                       float64
Politically_Exposed_Id         float64
Birth_

,ContractNumber,ProdCat,ProdCode,PolicyStatus,Switch_Flag,PricingVariant,Has_UWP,Insurance_Tax_Type,Main_Tax_Country,Secondary_Tax_Country,...,DATA_VERSION,POLICY_NUMBER,VN_MILIEU_CODE,VP_MILIEU_CODE,VN_TARGET_GROUP_CODE,VP_TARGET_GROUP_CODE,VN_MILIEU_DESCRIPTION,VN_TARGET_GROUP_DESCRIPTION,VP_MILIEU_DESCRIPTION,VP_TARGET_GROUP_DESCRIPTION
0,6074160W,XRPP,XRPP_2005V1,I,0,NaN,0,None,Deutschland,Deutschland,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6074937L,XRPP,XRPP_2005V1,I,0,NaN,0,None,Deutschland,Deutschland,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6074962M,XRPP,XRPP_2005V1,I,0,NaN,0,None,Deutschland,Deutschland,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6075850C,XRPP,XRPP_2005V1,F,0,NaN,0,None,Deutschland,Deutschland,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6100638P,XRPP,XRPP_2005V1,I,0,NaN,0,None,Deutschland,Deutschland,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6100746E,XRPP,XRPP_2005V1,I,0,NaN,0,None,Deutschland,Deutschland,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6214246Y,XRPP,XRPP_2005V1,I,0,NaN,0,None,Deutschland,Deutschland,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,6215630F,XRPP,XRPP_2005V1,I,0,NaN,0,None,Deutschland,Deutschland,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,6215740Y,XRPP,XRPP_2005V1,I,0,NaN,0,None,Deutschland,Deutschland,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,6215812A,XRPP,XRPP_2005V1,I,0,4015.0,1,None,Deutschland,Deutschland,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
df_Horizon_All_Features['Insurance_Tax_Type'] = df_Horizon_All_Features['Insurance_Tax_Type'].fillna(0).apply(np.int64)
print(df_Horizon_All_Features['Insurance_Tax_Type'].value_counts())

df_Horizon_All_Features = df_Horizon_All_Features.drop(['Insurance_Tax_Type'],axis=1)
df_Horizon_All_Features.head(20)
print(df_Horizon_All_Features.dtypes)


0    6186041
Name: Insurance_Tax_Type, dtype: int64
ContractNumber                  object
ProdCat                         object
ProdCode                        object
PolicyStatus                    object
Switch_Flag                      int64
PricingVariant                 float64
Has_UWP                          int64
Main_Tax_Country                object
Secondary_Tax_Country           object
Taxed_Retirement_Income        float64
Policy_Fee_Type_Desc            object
Future_Risk                     object
ServicingBroker                float64
InitialBroker                  float64
BrokerCategory                  object
InitialRate                    float64
ReneewalRate                   float64
Policy_Number                   object
VIP_Type_Id                    float64
Marital_Status_Id              float64
Nationality_Code                object
IsSmoker                       float64
Politically_Exposed_Id         float64
Birth_Place                     object
Address_Post

In [39]:
print(df_Horizon_All_Features['Main_Tax_Country'].value_counts())
df_Horizon_All_Features = df_Horizon_All_Features.drop(['Main_Tax_Country'],axis=1)
print(df_Horizon_All_Features.dtypes)

Deutschland     6186041
Name: Main_Tax_Country, dtype: int64
ContractNumber                  object
ProdCat                         object
ProdCode                        object
PolicyStatus                    object
Switch_Flag                      int64
PricingVariant                 float64
Has_UWP                          int64
Secondary_Tax_Country           object
Taxed_Retirement_Income        float64
Policy_Fee_Type_Desc            object
Future_Risk                     object
ServicingBroker                float64
InitialBroker                  float64
BrokerCategory                  object
InitialRate                    float64
ReneewalRate                   float64
Policy_Number                   object
VIP_Type_Id                    float64
Marital_Status_Id              float64
Nationality_Code                object
IsSmoker                       float64
Politically_Exposed_Id         float64
Birth_Place                     object
Address_Postcode                 int32
Sex

In [40]:
print(df_Horizon_All_Features['Secondary_Tax_Country'].value_counts())
df_Horizon_All_Features = df_Horizon_All_Features.drop(['Secondary_Tax_Country'],axis=1)
print(df_Horizon_All_Features.dtypes)


Deutschland     6186041
Name: Secondary_Tax_Country, dtype: int64
ContractNumber                  object
ProdCat                         object
ProdCode                        object
PolicyStatus                    object
Switch_Flag                      int64
PricingVariant                 float64
Has_UWP                          int64
Taxed_Retirement_Income        float64
Policy_Fee_Type_Desc            object
Future_Risk                     object
ServicingBroker                float64
InitialBroker                  float64
BrokerCategory                  object
InitialRate                    float64
ReneewalRate                   float64
Policy_Number                   object
VIP_Type_Id                    float64
Marital_Status_Id              float64
Nationality_Code                object
IsSmoker                       float64
Politically_Exposed_Id         float64
Birth_Place                     object
Address_Postcode                 int32
Sex_Id                         float6

In [41]:
print(df_Horizon_All_Features['Taxed_Retirement_Income'].value_counts())
df_Horizon_All_Features = df_Horizon_All_Features.drop(['Taxed_Retirement_Income'],axis=1)
print(df_Horizon_All_Features.dtypes)

0.0    5462198
Name: Taxed_Retirement_Income, dtype: int64
ContractNumber                  object
ProdCat                         object
ProdCode                        object
PolicyStatus                    object
Switch_Flag                      int64
PricingVariant                 float64
Has_UWP                          int64
Policy_Fee_Type_Desc            object
Future_Risk                     object
ServicingBroker                float64
InitialBroker                  float64
BrokerCategory                  object
InitialRate                    float64
ReneewalRate                   float64
Policy_Number                   object
VIP_Type_Id                    float64
Marital_Status_Id              float64
Nationality_Code                object
IsSmoker                       float64
Politically_Exposed_Id         float64
Birth_Place                     object
Address_Postcode                 int32
Sex_Id                         float64
Marital_Status                 float64
Pensi

In [42]:
print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')


2024-07-18 19:36:35.776029
svmem(total=16756752384, available=5573206016, percent=66.7, used=11183546368, free=5573206016)


# Checkpoint to reset the dataframes.

In [ ]:
#df_Horizon_All_Features['Lump_Sum_Taxation'] = df_Horizon_All_Features['Lump_Sum_Taxation'].fillna(0).apply(np.int64)
print(df_Horizon_All_Features['Lump_Sum_Taxation'].value_counts())

#df_Horizon_All_Features = df_Horizon_All_Features.drop(['Lump_Sum_Taxation'],axis=1)
#print(df_Horizon_All_Features.dtypes)
df_Horizon_All_Features.head(20)

In [ ]:
#df_Horizon_All_Features['Lump_Sum_Taxation'] = df_Horizon_All_Features['Lump_Sum_Taxation'].fillna(0).apply(np.int64)
print(df_Horizon_All_Features['Lump_Sum_Taxation'].value_counts())

#df_Horizon_All_Features = df_Horizon_All_Features.drop(['Lump_Sum_Taxation'],axis=1)
#print(df_Horizon_All_Features.dtypes)


In [69]:
df_Horizon_All_Features.head(20)

,ContractNumber,ProdCat,ProdCode,PolicyStatus,Switch_Flag,PricingVariant,Has_UWP,Taxed_Retirement_Income,Policy_Fee_Type_Desc,Future_Risk,...,DATA_VERSION,POLICY_NUMBER,VN_MILIEU_CODE,VP_MILIEU_CODE,VN_TARGET_GROUP_CODE,VP_TARGET_GROUP_CODE,VN_MILIEU_DESCRIPTION,VN_TARGET_GROUP_DESCRIPTION,VP_MILIEU_DESCRIPTION,VP_TARGET_GROUP_DESCRIPTION
0,6074160W,XRPP,XRPP_2005V1,I,0,NaN,0,0.0,Ein,kein,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6074937L,XRPP,XRPP_2005V1,I,0,NaN,0,0.0,Ein,kein,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6074962M,XRPP,XRPP_2005V1,I,0,NaN,0,0.0,Ein,kein,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6075850C,XRPP,XRPP_2005V1,F,0,NaN,0,0.0,Ein,kein,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6100638P,XRPP,XRPP_2005V1,I,0,NaN,0,0.0,Ein,kein,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6100746E,XRPP,XRPP_2005V1,I,0,NaN,0,0.0,Ein,kein,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6214246Y,XRPP,XRPP_2005V1,I,0,NaN,0,0.0,Ein,kein,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,6215630F,XRPP,XRPP_2005V1,I,0,NaN,0,0.0,Ein,kein,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,6215740Y,XRPP,XRPP_2005V1,I,0,NaN,0,0.0,Ein,kein,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,6215812A,XRPP,XRPP_2005V1,I,0,4015.0,1,0.0,Ein,kein,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Drop obsolete columns
#df_Horizon_with_Switches = df_Horizon_with_Switches.drop(['PROCESSING_DATE'],axis=1)

In [58]:
df_Target_Metrics_Binary_DataRobot = df_Final_Target_Metrics[['PosBookEvent_4','NegBookEvent_4','PremiumChangeAmt_4',
                                                              'Premium_4','IndexAmt_4','PolicyStatus_4','StatusChange_4',
                                                              'ServicingBroker_4','BrokerChange_4','InvRatioF_4','InvRatioU_4',
                                                              'PosBookEvent_3','NegBookEvent_3','PremiumChangeAmt_3',
                                                              'Premium_3','IndexAmt_3','PolicyStatus_3','StatusChange_3',
                                                              'ServicingBroker_3','BrokerChange_3','InvRatioF_3','InvRatioU_3',
                                                              'PosBookEvent_2','NegBookEvent_2','PremiumChangeAmt_2',
                                                              'Premium_2','IndexAmt_2','PolicyStatus_2','StatusChange_2',
                                                              'ServicingBroker_2','BrokerChange_2','InvRatioF_2','InvRatioU_2',
                                                              'PosBookEvent_1','NegBookEvent_1','PremiumChangeAmt_1',
                                                              'Premium_1','IndexAmt_1','PolicyStatus_1','StatusChange_1',
                                                              'ServicingBroker_1','BrokerChange_1','InvRatioF_1','InvRatioU_1',
                                                              'Premium_0','PolicyStatus_0','ServicingBroker_0',
                                                              'ProdCat','ProdCode','InitialBroker','PayFreq','PricingVariant',
                                                              'IndexationPercent','PaymentMethod','LumpSumTax','Anniversary_Period',
                                                              'Term','Term_Passed','Term_Remaining','TermRemainingLTEQ_5',
                                                              'TermRemainingLTEQ_10','Anniversary_Month','PremHolStartDt',
                                                              'PremHolEndDt','IndexationStartDt','Risk_Commencement_Date',
                                                              'pAV_2022_Campaign','VIP_Type_Id','Marital_Status_Id','Nationality_Code',
                                                              'Is_Smoker','Politically_Exposed_Id','Birth_Place','Sex_Id',
                                                              'Marital_Status','Pension_Disability_Level','WOP_Disability_Level',
                                                              'Address_City','Partner_Type_De','Is_EU_Sanctioned','Address_District',
                                                              'Age_Start_Horizon','Age_Start_Yr','Postal_Code','Postal_Code_Name',
                                                              'District','State','VN_MILIEU_CODE','VN_TARGET_GROUP_CODE',
                                                              'VN_MILIEU_DESCRIPTION','VP_MILIEU_DESCRIPTION']]

df_Target_Metrics_Binary_DataRobot.head(25)

,PosBookEvent_4,NegBookEvent_4,PremiumChangeAmt_4,Premium_4,IndexAmt_4,PolicyStatus_4,StatusChange_4,ServicingBroker_4,BrokerChange_4,InvRatioF_4,...,Age_Start_Horizon,Age_Start_Yr,Postal_Code,Postal_Code_Name,District,State,VN_MILIEU_CODE,VN_TARGET_GROUP_CODE,VN_MILIEU_DESCRIPTION,VP_MILIEU_DESCRIPTION
0,0,0,0.00,0.0,0.0,I,NaN,9285,0,NaN,...,51.343970,50.766272,85416.0,Langenbach,Landkreis Freising,Bavaria,07_TRA,Traditionelle,07_TRA,Traditionelle
1,0,0,0.00,0.0,0.0,I,NaN,25402,0,NaN,...,58.043629,57.465930,44229.0,Dortmund,Dortmund,North Rhine-Westphalia,02_PMA,AnsprRoutinierte,02_PMA,AnsprRoutinierte
2,0,0,0.00,0.0,0.0,I,NaN,23392,0,NaN,...,49.191975,48.614277,20255.0,Hamburg,Hamburg,Hamburg,06_NOB,PreissensPragm,06_NOB,PreissensPragm
3,0,0,0.00,0.0,0.0,R,NaN,2571,0,NaN,...,60.206575,59.628877,56154.0,Boppard,Landkreis Rhein-Hunsr�ck-Kreis,Rhineland-Palatinate,NaN,NaN,NaN,NaN
4,0,0,0.00,0.0,0.0,I,NaN,12310,0,NaN,...,72.431330,71.853631,70771.0,Leinfelden-Echterdingen,Landkreis Esslingen,Baden-W�rttemberg,01_KOG,AnsprRoutinierte,01_KOG,AnsprRoutinierte
5,0,0,0.00,0.0,0.0,I,NaN,9923,0,NaN,...,48.959253,48.381555,60486.0,Frankfurt am Main,Frankfurt am Main,Hesse,08_PRE,Traditionelle,08_PRE,Traditionelle
6,0,0,0.00,0.0,0.0,I,NaN,25402,0,NaN,...,62.331191,61.753493,63743.0,Aschaffenburg,Aschaffenburg,Bavaria,05_ADA,PreissensPragm,05_ADA,PreissensPragm
7,0,1,-2119.95,0.0,0.0,I,NaN,12371,0,NaN,...,64.047859,63.470160,70619.0,Stuttgart,Stuttgart,Baden-W�rttemberg,01_KOG,AnsprRoutinierte,01_KOG,AnsprRoutinierte
8,0,0,0.00,0.0,0.0,I,NaN,16586,0,NaN,...,53.342642,52.764944,92334.0,Berching,Landkreis Neumarkt in der Oberpfalz,Bavaria,06_NOB,Selbstoptimierer,03_PER,Selbstoptimierer
9,0,0,0.00,0.0,0.0,I,NaN,30466,0,NaN,...,54.388523,53.810824,32584.0,L�hne,Landkreis Herford,North Rhine-Westphalia,NaN,NaN,NaN,NaN


# Remove unused columns after merge.
df_Final_Target_Metrics = df_Final_Target_Metrics.drop(['Contract_Number','Anniversary_Date','Risk_Commencement_Date',
                                                        'Cover_Ends','Horizon_Start_Date','Year_Start_Date','Policy_Number',
                                                        'Address_Postcode','DOB','CONTRACT_NUMBER','DATA_VERSION',
                                                        'POLICY_NUMBER','VP_MILIEU_CODE','VP_TARGET_GROUP_CODE',
                                                        'VN_TARGET_GROUP_DESCRIPTION','VP_TARGET_GROUP_DESCRIPTION'], axis=1)

## Obfuscate the Postal_Code
df_Final_Target_Metrics.Postal_Code = df_Final_Target_Metrics.Postal_Code + 1234

In [61]:
df_Target_Metrics_Binary_DataRobot.head(5)

,PosBookEvent_4,NegBookEvent_4,PremiumChangeAmt_4,Premium_4,IndexAmt_4,PolicyStatus_4,StatusChange_4,ServicingBroker_4,BrokerChange_4,InvRatioF_4,...,Age_Start_Horizon,Age_Start_Yr,Postal_Code,Postal_Code_Name,District,State,VN_MILIEU_CODE,VN_TARGET_GROUP_CODE,VN_MILIEU_DESCRIPTION,VP_MILIEU_DESCRIPTION
0,0,0,0.0,0.0,0.0,I,NaN,9285,0,NaN,...,51.343970,50.766272,85416.0,Langenbach,Landkreis Freising,Bavaria,07_TRA,Traditionelle,07_TRA,Traditionelle
1,0,0,0.0,0.0,0.0,I,NaN,25402,0,NaN,...,58.043629,57.465930,44229.0,Dortmund,Dortmund,North Rhine-Westphalia,02_PMA,AnsprRoutinierte,02_PMA,AnsprRoutinierte
2,0,0,0.0,0.0,0.0,I,NaN,23392,0,NaN,...,49.191975,48.614277,20255.0,Hamburg,Hamburg,Hamburg,06_NOB,PreissensPragm,06_NOB,PreissensPragm
3,0,0,0.0,0.0,0.0,R,NaN,2571,0,NaN,...,60.206575,59.628877,56154.0,Boppard,Landkreis Rhein-Hunsr�ck-Kreis,Rhineland-Palatinate,NaN,NaN,NaN,NaN
4,0,0,0.0,0.0,0.0,I,NaN,12310,0,NaN,...,72.431330,71.853631,70771.0,Leinfelden-Echterdingen,Landkreis Esslingen,Baden-W�rttemberg,01_KOG,AnsprRoutinierte,01_KOG,AnsprRoutinierte


In [62]:
df_Target_Metrics_Binary_DataRobot.dtypes

PosBookEvent_4                int64
NegBookEvent_4                int64
PremiumChangeAmt_4          float64
Premium_4                   float64
IndexAmt_4                  float64
PolicyStatus_4               object
StatusChange_4               object
ServicingBroker_4             int64
BrokerChange_4                int64
InvRatioF_4                 float64
InvRatioU_4                 float64
PosBookEvent_3                int64
NegBookEvent_3                int64
PremiumChangeAmt_3          float64
Premium_3                   float64
IndexAmt_3                  float64
PolicyStatus_3               object
StatusChange_3               object
ServicingBroker_3             int64
BrokerChange_3                int64
InvRatioF_3                 float64
InvRatioU_3                 float64
PosBookEvent_2                int64
NegBookEvent_2                int64
PremiumChangeAmt_2          float64
Premium_2                   float64
IndexAmt_2                  float64
PolicyStatus_2              

# writing to Excel

In [63]:
# Write out to a comma separated values file.
filename = r'C:\Users\m033\OneDrive - GWLE\CLE Analytics\Propensity Models\Up-Sell Private Pension\Data\Target_Metrics_Binary_DataRobot.csv'
df_Target_Metrics_Binary_DataRobot.to_csv(filename, encoding='utf-8', index=False)        
df_Target_Metrics_Binary_DataRobot.head(10)

C:\Users\m033\Anaconda3\lib\site-packages\pandas\core\internals\blocks.py:2106: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)


,PosBookEvent_4,NegBookEvent_4,PremiumChangeAmt_4,Premium_4,IndexAmt_4,PolicyStatus_4,StatusChange_4,ServicingBroker_4,BrokerChange_4,InvRatioF_4,...,Age_Start_Horizon,Age_Start_Yr,Postal_Code,Postal_Code_Name,District,State,VN_MILIEU_CODE,VN_TARGET_GROUP_CODE,VN_MILIEU_DESCRIPTION,VP_MILIEU_DESCRIPTION
0,0,0,0.00,0.0,0.0,I,NaN,9285,0,NaN,...,51.343970,50.766272,85416.0,Langenbach,Landkreis Freising,Bavaria,07_TRA,Traditionelle,07_TRA,Traditionelle
1,0,0,0.00,0.0,0.0,I,NaN,25402,0,NaN,...,58.043629,57.465930,44229.0,Dortmund,Dortmund,North Rhine-Westphalia,02_PMA,AnsprRoutinierte,02_PMA,AnsprRoutinierte
2,0,0,0.00,0.0,0.0,I,NaN,23392,0,NaN,...,49.191975,48.614277,20255.0,Hamburg,Hamburg,Hamburg,06_NOB,PreissensPragm,06_NOB,PreissensPragm
3,0,0,0.00,0.0,0.0,R,NaN,2571,0,NaN,...,60.206575,59.628877,56154.0,Boppard,Landkreis Rhein-Hunsr�ck-Kreis,Rhineland-Palatinate,NaN,NaN,NaN,NaN
4,0,0,0.00,0.0,0.0,I,NaN,12310,0,NaN,...,72.431330,71.853631,70771.0,Leinfelden-Echterdingen,Landkreis Esslingen,Baden-W�rttemberg,01_KOG,AnsprRoutinierte,01_KOG,AnsprRoutinierte
5,0,0,0.00,0.0,0.0,I,NaN,9923,0,NaN,...,48.959253,48.381555,60486.0,Frankfurt am Main,Frankfurt am Main,Hesse,08_PRE,Traditionelle,08_PRE,Traditionelle
6,0,0,0.00,0.0,0.0,I,NaN,25402,0,NaN,...,62.331191,61.753493,63743.0,Aschaffenburg,Aschaffenburg,Bavaria,05_ADA,PreissensPragm,05_ADA,PreissensPragm
7,0,1,-2119.95,0.0,0.0,I,NaN,12371,0,NaN,...,64.047859,63.470160,70619.0,Stuttgart,Stuttgart,Baden-W�rttemberg,01_KOG,AnsprRoutinierte,01_KOG,AnsprRoutinierte
8,0,0,0.00,0.0,0.0,I,NaN,16586,0,NaN,...,53.342642,52.764944,92334.0,Berching,Landkreis Neumarkt in der Oberpfalz,Bavaria,06_NOB,Selbstoptimierer,03_PER,Selbstoptimierer
9,0,0,0.00,0.0,0.0,I,NaN,30466,0,NaN,...,54.388523,53.810824,32584.0,L�hne,Landkreis Herford,North Rhine-Westphalia,NaN,NaN,NaN,NaN


In [ ]:
%who_ls DataFrame 

In [ ]:
del df_premium

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')